# Imports

In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
from imutils.video import FileVideoStream
import cv2
import time
import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

filenames = glob.glob('/kaggle/input/deepfake-detection-challenge/test_videos/*.mp4')[:30]

In [ ]:
device

In [ ]:
!pip install /kaggle/input/facenet-pytorch-vggface2/facenet_pytorch-2.2.7-py3-none-any.whl

In [ ]:
!pip install /kaggle/input/d/meckdahl/imutils/imutils-0.5.3

# Haar Cascade

**Алгоритм:** Haar Cascade довольно простой алгоритм, использующий предподсчитанные featuremaps для пикселей различных соотношений на картинке, которые используются для обучения классификатора,  реализующегося в виде градиентного бустинга решающих деревьев. Поскольку сам предподсчет происходит динамически, он не занимает много времени, а потому главное преимущество Haar Cascade - его быстродействие.

In [ ]:
class HaarDetector():

    def __init__(self,faceCascadePath):
        self.faceCascade=cv2.CascadeClassifier(faceCascadePath)


    def detect(self, image, scaleFactor=1.1,
               minNeighbors=5,
               minSize=(5,5)):
        
        rects=self.faceCascade.detectMultiScale(image,
                                                scaleFactor=scaleFactor,
                                                minNeighbors=minNeighbors,
                                                minSize=minSize)
        return rects


In [ ]:
frontal_cascade_path=cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

fd=HaarDetector(frontal_cascade_path)

In [ ]:
def run_detection_haar(filenames, scaleFactor, minNeighbors, minSize):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()

    for filename in tqdm(filenames):

        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

        for j in range(v_len):

            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)

            if len(frames) >= batch_size or j == v_len - 1:
                faces = []
                for fr in frames:
                    faces.append(fd.detect(fr,
                               scaleFactor=scaleFactor,
                               minNeighbors=minNeighbors,
                               minSize=minSize))

                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []

                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )

        v_cap.stop()



In [ ]:
run_detection_haar(filenames, 
            scaleFactor=1.9, 
            minNeighbors=3, 
            minSize=(40,40))

Haar Cascade Detection - алгоритм, применяемый в случаях, когда возникает ограниченность вычислительных возможностей, но зачастую это дает просадку в качестве детекции, особенно на данных с низким разрешением. В нашем случае на full resolution видео модель обрабатывала в среднем ~35 кадров в секунду и отметила 8995 детектированных лиц.

# MTCNN

**Алгоритм:** модель использует три нейронных сети: P-Net, R-Net и O-Net, для качественного определения ограничительной рамки, а также положения лица в трехмерном пространстве за счет фичей положения глаз, носа и рта человека. На вход каждой из сетей подается Image Pyramid кандидатов ([см. статью](https://arxiv.org/ftp/arxiv/papers/1604/)), что позволяет очень точно определить как присутствие лица на изображении, так и координаты bounding box.

In [ ]:
class MTCNN_Detector(object):
    
    def __init__(self, stride=1, resize=1, *args, **kwargs):
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return faces

In [ ]:
mtcnn = MTCNN_Detector(
    stride=1,
    resize=1,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)

In [ ]:
def run_detection_mtcnn(mtcnn, filenames):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()

    for filename in tqdm(filenames):

        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

        for j in range(v_len):

            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

            if len(frames) >= batch_size or j == v_len - 1:

                faces = mtcnn(frames)

                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []

                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )

        v_cap.stop()


In [ ]:
run_detection_mtcnn(mtcnn, filenames)

MTCNN очень точно детектирует лица на отдельных изображениях, но из за обработки всех кадров подряд, теряет в скорости детекции. На нашем конкретном датасете MTCNN обрабатывала в среднем ~25 кадров в секунду и обнаружила 10978 лиц на всех кадрах.

# Fast MTCNN detector

**Алгоритм:** используемый алгоритм представляет собой модификацию MTCNN, в которой распознавание лиц выполняется только для каждых _N_ кадров и применяется ко всем кадрам. Например, с батчем из 9 кадров мы передаем кадры 0, 3 и 6 в MTCNN. Затем bounding boxes, возвращенные для кадра 0, будут просто применяться к кадрам 1 и 2. Аналогичным образом, обнаруженные лица для кадра 3 применяются к кадрам 4 и 5, а лица для кадров 6 применяются к кадрам 7. и 8.

Хотя алгоритм требует, чтобы лица не перемещались между кадрами существенно, это, как правило, хорошее приближение для небольшого количества страйдов. Если шаг равен 3, мы полагаем, что лицо существенно не меняет положение в течение дополнительных 2 кадров, или ~0,07 секунды. Если лица движутся быстрее, они, скорее всего, в любом случае будут очень размытыми. Более того, обеспечение обрезки граней с небольшими отступами снижает влияние смещения граней.

Класс ниже представляет собой оболочку оригинальной MTCNN из `facenet-pytorch` для реализации описанного выше алгоритма.

In [ ]:
class FastMTCNN(object):
    
    def __init__(self, stride, resize=1, *args, **kwargs):

        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):

        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return faces

In [ ]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)

In [ ]:
def run_detection_fmtcnn(fast_mtcnn, filenames):
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    start = time.time()

    for filename in tqdm(filenames):

        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

        for j in range(v_len):

            frame = v_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

            if len(frames) >= batch_size or j == v_len - 1:

                faces = fast_mtcnn(frames)

                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []

                print(
                    f'Frames per second: {frames_processed / (time.time() - start):.3f},',
                    f'faces detected: {faces_detected}\r',
                    end=''
                )

        v_cap.stop()

run_detection_fmtcnn(fast_mtcnn, filenames)


FastMTCNN значительно улучшила вычислительное время относительно MTCNN и качество детекции относительно HaarCascade со средней скоростью обработки видео ~90 кадров в секунду и 11037 детектированными лицами.

# Выводы

FastMTCNN показала наилучшие результаты на датасете с большим количеством людей в кадре по скорости обработки и точности детекции. Относительно исследованных моделей, данная является в разы превосходящей остальные.